In [9]:
import math
import time
import copy
from datetime import datetime
from pathlib import Path
import shutil
from glob import glob

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F

from PIL import Image
from tqdm.auto import tqdm

import pandas as pd
import numpy as np

METRIC_DIM = 512
IMG_SIZE = 112
device = torch.device("cuda:0")

In [10]:
# 훈련 이미지 리스트
BASE_DIR = '/data/kts123/aihub/reid'
train_imgs = '/data/kts123/aihub/reid/img_list_train.txt'

In [ ]:
df = pd.read_csv(train_imgs)
for kls, df_i in tqdm(df.groupby('KLS_IDX')):
    dst_dir = f'train_imgs/{kls:05d}'
    Path(dst_dir).mkdir(exist_ok=True, parents=True)
    for name in df_i['NAME'].values:
        src = f'{BASE_DIR}/{name}'
        dst = f'{dst_dir}/{Path(name).name}'
        shutil.copy(src, dst)

In [14]:
#val 이미지 리스트
val_imgs = '/data/kts123/aihub/reid/img_list_val.txt'
df = pd.read_csv(train_imgs)

In [ ]:
for kls, df_i in tqdm(df.groupby('KLS_IDX')):
    dst_dir = f'val_imgs/{kls:05d}'
    Path(dst_dir).mkdir(exist_ok=True, parents=True)
    for name in df_i['NAME'].values:
        src = f'{BASE_DIR}/{name}'
        dst = f'{dst_dir}/{Path(name).name}'
        shutil.copy(src, dst)

  0%|          | 0/502 [00:00<?, ?it/s]

In [1]:
from glob import glob
from torch.utils.data import Dataset, DataLoader

class FolderDataset(Dataset):
    def __init__(self, root_dir, phase = 'train', trsf = lambda e: e):
        imgs_total = glob(f'{root_dir}/*/*.jpg')
        imgs_train = [e for e in imgs_total if int(Path(e).stem) < 100]
        imgs_val   = [e for e in imgs_total if int(Path(e).stem) >= 100]
        self.imgs = imgs_train if phase == 'train' else imgs_val
        
        self.class_names = set([p.split('/')[-2] for p in self.imgs])
        self.class_names = sorted(list(self.class_names))
        self.labels = [self.class_names.index(e.split('/')[-2]) for e in self.imgs]
        self.trsf = trsf
        
    def num_labels(self):
        return len(self.class_names)
    
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, idx):
        path = self.imgs[idx]
        label = self.labels[idx]
        im = Image.open(path)
        im = self.trsf(im)
        return im, label
    
    

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(int(IMG_SIZE*1.2)),
        transforms.RandomResizedCrop(size=IMG_SIZE, scale=(0.64, 1.0), ratio=(0.9, 1.1)),
        transforms.ColorJitter(brightness=.5, hue=.3),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.RandomErasing(p=0.5, scale=(0.05, 0.1), value=0.5),
        transforms.RandomErasing(p=0.5, scale=(0.05, 0.1), value=0.5),
        transforms.RandomErasing(p=0.5, scale=(0.05, 0.1), value=0.5),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(int(IMG_SIZE*1.2)),
        transforms.CenterCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [ ]:
datasets =  {phase:FolderDataset(CROP_IMG_DIR, phase, data_transforms[phase]) 
             for phase in ['train', 'val']}

In [ ]:
dataloaders = {phase: DataLoader(datasets[phase], batch_size=64, 
                             shuffle=True, num_workers=2, drop_last=True)
               for phase in ['train', 'val']}

len(dataloaders['train']), len(dataloaders['val']), datasets['train'].num_labels()

In [ ]:
class ArcMarginProduct(nn.Module):
    def __init__(self, num_cls):
        super(ArcMarginProduct, self).__init__()
        self.num_cls = num_cls
        self.s       = 30 
        self.m       = 0.50 

        self.weight = Parameter(torch.FloatTensor(num_cls, METRIC_DIM))
        nn.init.xavier_uniform_(self.weight)

        self.cos_m = math.cos(self.m)
        self.sin_m = math.sin(self.m)
        self.th = math.cos(math.pi - self.m)
        self.mm = math.sin(math.pi - self.m) * self.m
        
    def forward(self, input_features, labels):
        cosine = F.linear(F.normalize(input_features), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))

        #cos(a + self.m) = cos(a)*cos(self.m) - sin(a)*sin(self.m)
        phi = cosine * self.cos_m - sine * self.sin_m
        phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        # convert label to one-hot
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, labels.view(-1, 1).long(), 1)

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [ ]:
class TrainModel(nn.Module):
    def __init__(self, num_classes):
        super(TrainModel, self).__init__()
        backbone = models.resnext50_32x4d(pretrained=True)
        backbone.fc = nn.Linear(in_features=backbone.fc.in_features, out_features=METRIC_DIM)
        self.backbone = backbone
        self.metric = ArcMarginProduct(num_classes)
        
    def forward(self, input_imgs, labels):
        x = self.backbone(input_imgs)
        x = self.metric(x, labels)
        return x
    
model = TrainModel(datasets['train'].num_labels())      

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc, best_epoch = 0.0, 0

    for epoch in tqdm(range(num_epochs), desc='epochs'):

        ds_size = 0
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            model.train(phase == 'train') # Set model to training/evaluate mode

            running_loss, running_corrects = 0.0, 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase], desc=f'{phase} ep({epoch:03d})'):
                inputs = inputs.to(device)
                labels = labels.to(device)
                ds_size += labels.size(0)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward: track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs, labels)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / ds_size
            epoch_acc = running_corrects.double() / ds_size

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc, best_epoch = epoch_acc, epoch
                best_model_wts = copy.deepcopy(model.state_dict())
        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc, best_epoch
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Decay LR by a factor of 0.1 every 5 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)